In [30]:
import numpy as np

import xobjects as xo
import xtrack as xt
import xpart as xp

## Generate a simple line
line = xt.Line(
    elements=[xt.Drift(length=2.),
              xt.Multipole(knl=[0, 1.], ksl=[0,0]),
              xt.Drift(length=1.),
              xt.Multipole(knl=[0, -1.], ksl=[0,0])],
    element_names=['drift_0', 'quad_0', 'drift_1', 'quad_1'])

## Attach a reference particle to the line (optional)
## (defines the reference mass, charge and energy)
line.particle_ref = xp.Particles(p0c=6500e9, #eV
                                 q0=1, mass0=xp.PROTON_MASS_EV)

## Choose a context
context = xo.ContextCpu()         # For CPU
# context = xo.ContextCupy()      # For CUDA GPUs
# context = xo.ContextPyopencl()  # For OpenCL GPUs

## Transfer lattice on context and compile tracking code
line.build_tracker(_context=context)

## Build particle object on context
n_part = 200
particles = xp.Particles(p0c=6500e9, #eV
                        q0=1, mass0=xp.PROTON_MASS_EV,
                        x=np.random.uniform(-1e-3, 1e-3, n_part),
                        px=np.random.uniform(-1e-5, 1e-5, n_part),
                        y=np.random.uniform(-2e-3, 2e-3, n_part),
                        py=np.random.uniform(-3e-5, 3e-5, n_part),
                        zeta=np.random.uniform(-1e-2, 1e-2, n_part),
                        delta=np.random.uniform(-1e-4, 1e-4, n_part),
                        _context=context)

## Track (saving turn-by-turn data)
n_turns = 100
line.track(particles, num_turns=n_turns,
              turn_by_turn_monitor=True)

## Turn-by-turn data is available at:
xData = np.array(list(line.record_last_track.x)).T
pxData = np.array(list(line.record_last_track.px)).T
# etc...

Found suitable prebuilt kernel `default_only_xtrack`.


In [35]:
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
import gc
%matplotlib qt
# import warnings
# import matplotlib.cbook
# warnings.filterwarnings("ignore",category=matplotlib.cbook.MatplotlibDeprecationWarning)


fig, ax = plt.subplots()
init_turn = 0
plot_phase, = ax.plot(xData[init_turn], pxData[init_turn], 'o')
ax.set_xlim([-5e-3, 5e-3])
ax.set_ylim([-1e-3, 1e-3])
fig.subplots_adjust(left=0.25, bottom=0.25)
ax_turn_slider = fig.add_axes([0.25, 0.1, 0.65, 0.03])
turn_slider = Slider(ax_turn_slider, "turns", 0, n_turns - 1, init_turn, valstep = 1)

def update(val):
    plot_phase.set_xdata(xData[turn_slider.val])
    plot_phase.set_ydata(pxData[turn_slider.val])
    fig.canvas.draw_idle()

turn_slider.on_changed(update)
plt.show()

gc.collect()

/var/folders/r3/ccbllzdn2pb3wzn_b988y1n80000gp/T/ipykernel_45211/1910802291.py:17: MatplotlibDeprecationWarning: Passing the valinit parameter of __init__() positionally is deprecated since Matplotlib 3.7; the parameter will become keyword-only two minor releases later.
  turn_slider = Slider(ax_turn_slider, "turns", 0, n_turns - 1, init_turn, valstep = 1)


8981

In [36]:
gc.collect()

10061